In [34]:
import tensorflow_decision_forests as tfdf
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import tempfile
import os

In [4]:
# Load a dataset
data = pd.read_csv("credit_card_transactions-ibm_v2.csv")

In [5]:
data.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No


In [26]:
# Split the data set into training and testing dataset
data.rename(columns={'Is Fraud?': 'Fraud'}, inplace=True)
train_data, test_data = train_test_split(data, test_size=0.3, random_state=1)
train_data.info()
print(train_data.columns[14])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17070830 entries, 16505099 to 12710949
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   User            int64  
 1   Card            int64  
 2   Year            int64  
 3   Month           int64  
 4   Day             int64  
 5   Time            object 
 6   Amount          object 
 7   Use Chip        object 
 8   Merchant Name   int64  
 9   Merchant City   object 
 10  Merchant State  object 
 11  Zip             float64
 12  MCC             int64  
 13  Errors?         object 
 14  Fraud           object 
dtypes: float64(1), int64(7), object(7)
memory usage: 2.0+ GB
Fraud


In [27]:
# Convert to TensorFlow dataset
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_data, label="Fraud")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data, label="Fraud")

In [28]:
# Train a Random Forest model.
model = tfdf.keras.RandomForestModel()
model.fit(train_ds)

Use /var/folders/ry/q793pzpj7j34b39xk_gqz14h0000gn/T/tmpwum_5_q4 as temporary training directory
Reading training dataset...
Training dataset read in 0:01:07.980536. Found 17070830 examples.
Training model...


[INFO 24-07-07 18:17:40.3026 AWST kernel.cc:1233] Loading model from path /var/folders/ry/q793pzpj7j34b39xk_gqz14h0000gn/T/tmpwum_5_q4/model/ with prefix f524ac00ebf842b5


Model trained in 0:44:26.889705
Compiling model...


[INFO 24-07-07 18:17:42.1220 AWST decision_forest.cc:734] Model loaded with 300 root(s), 963624 node(s), and 14 input feature(s).
[INFO 24-07-07 18:17:42.1220 AWST abstract_model.cc:1344] Engine "RandomForestGeneric" built
[INFO 24-07-07 18:17:42.1220 AWST kernel.cc:1061] Use fast generic engine


Model compiled.


In [29]:
# Summary of the model structure.
model.summary()

Model: "random_forest_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (14):
	Amount
	Card
	Day
	Errors_
	MCC
	Merchant_City
	Merchant_Name
	Merchant_State
	Month
	Time
	Use_Chip
	User
	Year
	Zip

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1. "Merchant_State"  0.234148 ################
    2.           "Year"  0.168253 #########
    3.            "MCC"  0.157392 ########
    4.  "Merchant_City"  0.155786 ########
    5.       "Use_Chip"  0.144665 #######
    6.         "Amount"  0.101968 ##
    7.            "Zip"  0.100596 ##
    8.  "Merchant_Name"  0.092202 #
    9.           "Time"  0.086393 #
   10.           "Card"  0.076642 
   11.          

In [30]:
# Compute model accuracy.
model.compile(metrics=["accuracy"])
model.evaluate(test_ds, return_dict=True)

7317/7317 [==============================] - 455s 62ms/step - loss: 0.0000e+00 - accuracy: 0.9995


{'loss': 0.0, 'accuracy': 0.9995375871658325}

In [44]:
model.save("1/")

INFO:tensorflow:Assets written to: 1/assets


INFO:tensorflow:Assets written to: 1/assets
